In [13]:
%run Package_AND_Parameter.ipynb

In [14]:
def AutoCorr(data, k=1):

    data = data.values
    m = np.mean(data)
    N = len(data)

    res = np.sum([(data[i + k] - m) * (data[i] - m)
                  for i in range(N - k)]) / np.sum([(data[i + k] - m)**(2)
                                                    for i in range(N - k)])
    return res


def ThresholdVol(data, thr, mode='nab_beq'):
    data = data.values

    return {
        'ab_beq': lambda x: len(x[abs(x) >= thr]) + 1,
        'ab_s': lambda x: len(x[abs(x) < thr]) + 1,
        'nab_beq': lambda x: len(x[x >= thr]) + 1,
        'nab_s': lambda x: len(x[x < thr]) + 1
    }[mode](data)


def Power(data, sfreq):

    data = data.values
    freq, psd = signal.welch(data, sfreq)

    return sum(psd)


def Mean_frequency(data, sfreq=100):

    data = data.values
    freq, psd = signal.welch(data, sfreq)

    return np.dot(freq, psd) / sum(psd)

def Spect_slop(data,sfreq=100):
    
    data = data.values
    freq, psd = signal.welch(data, sfreq)
    slope, intercept, r_value, p_value, slope_std_error = stats.linregress(freq, psd)
    
    return slope

# TEST CODE

df = pd.read_csv('test.csv')
df.head()